In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import calitp_data_analysis.magics

from rt_analysis import rt_filter_map_plot
import build_speedmaps_index

from IPython.display import display, Markdown
import pandas as pd
import geopandas as gpd
from siuba import *

from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    PROJECT_CRS,
    SEGMENT_GCS,
)

import shared_utils

In [15]:
## parameters cell
itp_id = 300

In [16]:
analysis_date = build_speedmaps_index.ANALYSIS_DATE
import datetime as dt
# if itp_id == 293:
#     analysis_date = dt.date(2023, 9, 13)

In [17]:
speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_itp_id == itp_id)

In [18]:
speedmap_index

,organization_name,organization_itp_id,organization_source_record_id,caltrans_district,_is_current,analysis_date,schedule_gtfs_dataset_key,status
53,City of Santa Monica,300,recJHFDLpGRMIFgnL,07 - Los Angeles,True,2024-04-17,efbbd5293be71f7a5de0cf82b59febe1,map_confirmed


In [19]:
SEGMENT_GCS

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/'

In [20]:
# speedmap_segs = gpd.read_parquet(f'{SEGMENT_GCS}segment_options/speedmap_segments_{analysis_date}.parquet')` #  non-aggregated

In [21]:
speedmap_segs = gpd.read_parquet(f'{SEGMENT_GCS}rollup_singleday/speeds_shape_speedmap_segments_{analysis_date}.parquet') #  aggregated

In [22]:
speedmap_segs.columns

Index(['schedule_gtfs_dataset_key', 'shape_array_key', 'shape_id', 'stop_pair',
       'stop_pair_name', 'segment_id', 'time_period', 'p50_mph', 'n_trips',
       'p20_mph', 'p80_mph', 'name', 'caltrans_district',
       'organization_source_record_id', 'organization_name', 'base64_url',
       'geometry'],
      dtype='object')

In [23]:
speedmap_segs = speedmap_segs >> filter(_.schedule_gtfs_dataset_key == speedmap_index.schedule_gtfs_dataset_key.iloc[0])

In [24]:
speedmap_segs >> head(3)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,stop_pair,stop_pair_name,segment_id,time_period,p50_mph,n_trips,p20_mph,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry
450953,efbbd5293be71f7a5de0cf82b59febe1,7952ac84db347ba2c6e65e60e5988275,26757,1640__1709,FLORENCE EB & LA CIENEGA NS__MESMER SB & JEFFE...,1640-1709-1,offpeak,15.54,16,12.64,17.86,Big Blue Bus Schedule,07 - Los Angeles,recJHFDLpGRMIFgnL,City of Santa Monica,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vY3VycmVudC...,"LINESTRING (-118.37135 33.96308, -118.37057 33..."
450954,efbbd5293be71f7a5de0cf82b59febe1,7952ac84db347ba2c6e65e60e5988275,26757,1640__1709,FLORENCE EB & LA CIENEGA NS__MESMER SB & JEFFE...,1640-1709-1,peak,12.75,14,9.04,18.41,Big Blue Bus Schedule,07 - Los Angeles,recJHFDLpGRMIFgnL,City of Santa Monica,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vY3VycmVudC...,"LINESTRING (-118.37135 33.96308, -118.37057 33..."
450955,efbbd5293be71f7a5de0cf82b59febe1,7952ac84db347ba2c6e65e60e5988275,26757,1640__1709,FLORENCE EB & LA CIENEGA NS__MESMER SB & JEFFE...,1640-1709-1,all_day,15.12,30,10.15,18.14,Big Blue Bus Schedule,07 - Los Angeles,recJHFDLpGRMIFgnL,City of Santa Monica,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vY3VycmVudC...,"LINESTRING (-118.37135 33.96308, -118.37057 33..."


In [25]:
speedmap_segs.time_period.unique()

array(['offpeak', 'peak', 'all_day'], dtype=object)

In [26]:
peak_test = speedmap_segs >> filter(_.time_period == 'peak')

In [27]:
time_periods = [(x, shared_utils.rt_utils.categorize_time_of_day(x)) for x in range(1, 25)]

In [47]:
time_periods

[(1, 'Owl'),
 (2, 'Owl'),
 (3, 'Owl'),
 (4, 'Early AM'),
 (5, 'Early AM'),
 (6, 'Early AM'),
 (7, 'AM Peak'),
 (8, 'AM Peak'),
 (9, 'AM Peak'),
 (10, 'Midday'),
 (11, 'Midday'),
 (12, 'Midday'),
 (13, 'Midday'),
 (14, 'Midday'),
 (15, 'PM Peak'),
 (16, 'PM Peak'),
 (17, 'PM Peak'),
 (18, 'PM Peak'),
 (19, 'PM Peak'),
 (20, 'Evening'),
 (21, 'Evening'),
 (22, 'Evening'),
 (23, 'Evening'),
 (24, 'Evening')]

In [28]:
total_peak = len([x[1] for x in time_periods if 'Peak' in x[1]])

In [29]:
total_peak

8

In [30]:
def prep_gdf(gdf):
    
    gdf = gdf.to_crs(calitp_data_analysis.geography_utils.CA_NAD83Albers)
    gdf['trips_per_hour'] = gdf.n_trips / total_peak

    ## shift to right side of road to display direction
    gdf.geometry = gdf.geometry.apply(shared_utils.rt_utils.try_parallel)
    gdf = gdf.apply(shared_utils.rt_utils.arrowize_by_frequency, axis=1)

    gdf = gdf >> arrange(_.trips_per_hour)

    return gdf

In [41]:
peak_test = prep_gdf(peak_test)

In [42]:
cmap = shared_utils.rt_utils.ZERO_THIRTY_COLORSCALE
url = shared_utils.rt_utils.SPEEDMAP_LEGEND_URL

In [45]:
speedmap_state = shared_utils.rt_utils.set_state_export(peak_test, filename='testnew5', map_type='speedmap',
                                      color_col='p20_mph', cmap=cmap, legend_url=url,
                                              cache_seconds=0, map_title=f'Speedmap Segs {analysis_date}')

writing to calitp-map-tiles/testing/testnew5.geojson.gz


In [46]:
speedmap_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Speedmap Segs 2024-04-17',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/testnew5.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (34.026346806862165, -118.44342716777291),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTcGVlZG1hcCBTZWdzIDIwMjQtMDQtMTciLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvdGVzdG5ldzUuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41LCAidG9vbHRpcF9zcGVlZF9rZXkiOiAicDIwX21waCJ9LCAidHlwZSI6ICJzcGVlZG1hcCJ9XSwgImxhdF9sb24iOiBbMzQuMDI2MzQ2ODA2ODYyMTY1LCAtMTE4LjQ0MzQyNzE2Nzc3MjkxXSwgInpvb20iOiAxMywgImxlZ2VuZF

## TODO replace status with a scan of speedmap segs column?